In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style("dark")
import time

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from qiskit import *
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, RealAmplitudes, ZFeatureMap, ZZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import CCXGate, CRZGate, RXGate
from qiskit.algorithms.optimizers import COBYLA, SLSQP, SPSA
from matplotlib import pyplot as plt
from IPython.display import clear_output
from qiskit.primitives import Sampler
from qiskit.circuit import ParameterVector
from sklearn.metrics import recall_score, precision_score, f1_score
import time
from qiskit_machine_learning.algorithms.classifiers import VQC
import os

In [ ]:
train = pd.read_csv("../data/train_fe.csv")

In [ ]:
cols = ['total_day_minutes', 'total_day_calls','total_intl_charge', 'customer_service_calls', 'account_length','number_vmail_messages', 
        'region_South', 'region_West']

In [ ]:
#train = train[cols]
x_train = train[cols]
y_train = train["churn"]

In [ ]:
sizes = np.linspace(0.1,1,10)
sizes = list(sizes)
sizes

In [ ]:
cols[:len(cols)+1]

In [ ]:
sizes[0]

In [ ]:
numbers = np.random.randint(0,high=len(train), size=round(len(train)*sizes[0]))

In [ ]:
np.array(y_train.iloc[numbers].reset_index(drop=True))

In [ ]:
size = pd.DataFrame()
for i in range(len(sizes)):
    numbers = np.random.randint(0,high=len(train), size=round(len(train)*sizes[i]))
    new_x = x_train.iloc[numbers,:].reset_index(drop=True)
    new_y = np.array(y_train.iloc[numbers].reset_index(drop=True))
    pauli_feature_map = PauliFeatureMap(feature_dimension=len(new_x.columns), reps=1)
    sampler = Sampler()
    ansatz_su = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry", "rz"], entanglement= "full",
                         insert_barriers=True)
    num_iter=300
    cobyla = COBYLA(maxiter = num_iter)
    model = VQC(
        sampler=sampler,
        feature_map=pauli_feature_map,
        ansatz=ansatz_su,
        optimizer=cobyla)
    new_x_arr = new_x.to_numpy()
    #new_y_arr = new_y.to_numpy()
    start = time.time()
    model.fit(new_x_arr, new_y)
    stop = time.time()
    elapsed=stop-start
    size.loc[i, "size"] = sizes[i]*len(train)
    size.loc[i, "model"] = "VQC_best"
    size.loc[i, "runtime"] = elapsed
    #size.loc[i,"kernel"] = typ
    print(i)

In [ ]:
feat = pd.DataFrame()
for i in range(1,len(cols)):
    new_x = x_train.loc[:,cols[:i+1]]
    new_x_arr = new_x.to_numpy()
    new_y = np.array(y_train)
    pauli_feature_map = PauliFeatureMap(feature_dimension=len(new_x.columns), reps=1)
    sampler = Sampler()
    if len(x_train.columns) == 1:
        ansatz_su = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry"],
                             insert_barriers=True)
    else: 
        ansatz_su = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry", "rz"], entanglement= "full",
                             insert_barriers=True)
    num_iter=300
    cobyla = COBYLA(maxiter = num_iter)
    model = VQC(
        sampler=sampler,
        feature_map=pauli_feature_map,
        ansatz=ansatz_su,
        optimizer=cobyla)
    start = time.time()
    model.fit(new_x_arr, new_y)
    stop = time.time()
    elapsed=stop-start
    feat.loc[i, "num_features"] = i + 1
    feat.loc[i, "model"] = "VQC_best"
    feat.loc[i, "runtime"] = elapsed
    #feat.loc[i,"kernel"] = typ

In [ ]:
num_iter=300
sampler = Sampler()
cobyla = COBYLA(maxiter = num_iter)
spsa = SPSA(maxiter = num_iter)

In [ ]:
def vqc_runtime(ansatz, optimizer):
    size = pd.DataFrame()
    for i in range(len(sizes)):
        numbers = np.random.randint(0,high=len(train), size=round(len(train)*sizes[i]))
        new_x = x_train.iloc[numbers,:].reset_index(drop=True)
        new_y = np.array(y_train.iloc[numbers].reset_index(drop=True))
        pauli_feature_map = PauliFeatureMap(feature_dimension=len(new_x.columns), reps=1)
        if optimizer == "cobyla":
            optim_use = cobyla
        elif optimizer == "spsa":
            optim_use = spsa

        if ansatz == "su2":
            ansatz_use = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry", "rz"],
                                      entanglement= "full", insert_barriers=True)
        elif ansatz == "two_local":
            ansatz_use = ansatz_two_local = TwoLocal(num_qubits=pauli_feature_map.width(),rotation_blocks=["ry", "rz"],
                                                     entanglement_blocks="cx", entanglement="linear", reps=2,
                                                     insert_barriers=True)
        elif ansatz == "n_local":
            #ansatz_use = ansatz_n_local
            theta = Parameter("θ")
            ansatz_use = NLocal(num_qubits=pauli_feature_map.width(),rotation_blocks=[RXGate(theta), CRZGate(theta)],
                        entanglement_blocks=CCXGate(),
                        entanglement=[[0, 1, 2], [0,2,1]],reps=2,insert_barriers=True)


        ansatz = ansatz_use
        model = VQC(
            sampler=sampler,
            feature_map=pauli_feature_map,
            ansatz=ansatz,
            optimizer=optim_use)
        new_x_arr = new_x.to_numpy()
        #new_y_arr = new_y.to_numpy()
        start = time.time()
        model.fit(new_x_arr, new_y)
        stop = time.time()
        elapsed=stop-start
        size.loc[i, "size"] = sizes[i]*len(train)
        size.loc[i, "model"] = "VQC" + "_" + str(ansatz) + "_" + str(optimizer)
        size.loc[i, "runtime"] = elapsed
        #size.loc[i,"kernel"] = typ
    size.to_csv("../vqc_results/runtime_size/vqc" + "_" + str(ansatz) + "_" + str(optimizer) + ".csv", index=False)
    feat = pd.DataFrame()
    for i in range(1,len(cols)):
        new_x = x_train.loc[:,cols[:i+1]]
        new_x_arr = new_x.to_numpy()
        new_y = np.array(y_train)
        pauli_feature_map = PauliFeatureMap(feature_dimension=len(new_x.columns), reps=1)
        if optimizer == "cobyla":
            optim_use = cobyla
        elif optimizer == "spsa":
            optim_use = spsa

        if ansatz == "su2":
            ansatz_use = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry", "rz"], entanglement= "full",
                             insert_barriers=True)
        elif ansatz == "two_local":
            ansatz_use = ansatz_two_local = TwoLocal(num_qubits=pauli_feature_map.width(),rotation_blocks=["ry", "rz"],
                                                     entanglement_blocks="cx", entanglement="linear", reps=2,
                                                     insert_barriers=True)
        elif ansatz == "n_local":
            #ansatz_use = ansatz_n_local
            theta = Parameter("θ")
            ansatz_use = NLocal(num_qubits=pauli_feature_map.width(),rotation_blocks=[RXGate(theta), CRZGate(theta)],
                        entanglement_blocks=CCXGate(),
                        entanglement=[[0, 1, 2], [0,2,1]],reps=2,insert_barriers=True)


        ansatz = ansatz_use
        model = VQC(
            sampler=sampler,
            feature_map=pauli_feature_map,
            ansatz=ansatz_su,
            optimizer=cobyla)
        start = time.time()
        model.fit(new_x_arr, new_y)
        stop = time.time()
        elapsed=stop-start
        feat.loc[i, "num_features"] = i + 1
        feat.loc[i, "model"] = "VQC" + "_" + str(ansatz) + "_" + str(optimizer)
        feat.loc[i, "runtime"] = elapsed
        #feat.loc[i,"kernel"] = typ
    feat.to_csv("../vqc_results/runtime_features/vqc" + "_" + str(ansatz) + "_" + str(optimizer) + ".csv", index=False)

In [ ]:
optim = ["cobyla", "spsa"]
ansatz_list = ["su2", "two_local", "n_local"]
for i in range(len(optim)):
    for j in range(len(ansatz_list)):
        print(optim[i], ansatz_list[j])
        vqc_runtime(ansatz_list[j], optim[i])